In [ ]:
import tensorflow
import matplotlib.pyplot as plt
import numpy as np
import scipy
import cv2
import os
from glob import glob
import shutil

In [ ]:
def apply_sobel(image):
    grad_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    abs_grad_x = cv2.convertScaleAbs(grad_x)
    abs_grad_y = cv2.convertScaleAbs(grad_y)
    return cv2.addWeighted(abs_grad_x, 0.5, abs_grad_y, 0.5, 0)

imagem = "/pesquisa/deteccao_arvores/yolo_v8/new_dataset/v6/dataset/train/images/0.jpg"
plt.imshow(apply_sobel(imagem))
plt.colorbar()

In [ ]:
# Dataset train
input_image_directory = '/pesquisa/deteccao_arvores/yolo_v8/new_dataset/v6/dataset/train/images'
input_label_directory = '/pesquisa/deteccao_arvores/yolo_v8/new_dataset/v6/dataset/train/labels'
output_image_directory = '/pesquisa/deteccao_arvores/yolo_v8/new_dataset/v6/dataset/train/images'
output_label_directory = '/pesquisa/deteccao_arvores/yolo_v8/new_dataset/v6/dataset/train/labels'

In [ ]:
def apply_canny(image):
    return cv2.Canny(image, 100, 200)

def apply_prewitt(image):
    kernelx = np.array([[1, 1, 1], [0, 0, 0], [-1, -1, -1]])
    kernely = np.array([[1, 0, -1], [1, 0, -1], [1, 0, -1]])
    img_prewittx = cv2.filter2D(image, -1, kernelx)
    img_prewitty = cv2.filter2D(image, -1, kernely)
    return img_prewittx + img_prewitty

def apply_sobel(image):
    grad_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    abs_grad_x = cv2.convertScaleAbs(grad_x)
    abs_grad_y = cv2.convertScaleAbs(grad_y)
    return cv2.addWeighted(abs_grad_x, 0.5, abs_grad_y, 0.5, 0)

def augment_images(input_image_dir, input_label_dir, output_image_dir, output_label_dir, technique):
    if not os.path.exists(input_image_dir):
        print(f"O diretório de entrada das imagens {input_image_dir} não existe.")
        return
    if not os.path.exists(input_label_dir):
        print(f"O diretório de entrada dos labels {input_label_dir} não existe.")
        return
    if not os.path.exists(output_image_dir):
        os.makedirs(output_image_dir)
        print(f"O diretório de saída das imagens {output_image_dir} foi criado.")
    if not os.path.exists(output_label_dir):
        os.makedirs(output_label_dir)
        print(f"O diretório de saída dos labels {output_label_dir} foi criado.")

    image_paths = glob(os.path.join(input_image_dir, '*.jpg'))
    if not image_paths:
        print("Nenhuma imagem .jpg encontrada no diretório de entrada das imagens.")
        return

    for image_path in image_paths:
        try:
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            if image is None:
                print(f"Não foi possível ler a imagem: {image_path}")
                continue

            base_name = os.path.basename(image_path)
            # Verificar se a imagem já foi processada com qualquer técnica
            if any(prefix in base_name for prefix in ['canny_', 'prewitt_', 'sobel_']):
                print(f">>> A imagem {base_name} já foi processada com alguma técnica.")
                continue

            if technique == 'canny':
                augmented_image = apply_canny(image)
            elif technique == 'prewitt':
                augmented_image = apply_prewitt(image)
            elif technique == 'sobel':
                augmented_image = apply_sobel(image)
            else:
                raise ValueError("Technique not recognized. Choose from 'canny', 'prewitt', or 'sobel'.")

            output_image_path = os.path.join(output_image_dir, f'{technique}_' + base_name)
            cv2.imwrite(output_image_path, augmented_image)
            print(f"Imagem processada salva em: {output_image_path}")

            # Construir o caminho do arquivo .txt correspondente
            txt_path = os.path.join(input_label_dir, os.path.splitext(base_name)[0] + '.txt')
            if os.path.exists(txt_path):
                # Construir o caminho de saída para o arquivo .txt
                output_txt_path = os.path.join(output_label_dir, f'{technique}_' + os.path.basename(txt_path))
                # Copiar o arquivo .txt para o diretório de saída
                shutil.copy(txt_path, output_txt_path)
                print(f"Arquivo .txt copiado para: {output_txt_path}")
            else:
                print(f"Arquivo .txt não encontrado para a imagem: {image_path}")

        except Exception as e:
            print(f"Erro ao processar a imagem {image_path}: {e}")

# Aplicar técnica específica
# technique = 'prewitt'  # Pode ser 'canny', 'prewitt' ou 'sobel'
technique = 'sobel'  # Pode ser 'canny', 'prewitt' ou 'sobel'

augment_images(input_image_directory, input_label_directory, output_image_directory, output_label_directory, technique)

